In [91]:
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
import csv
import re
import joblib
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import lightgbm as lgb
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import tensorflow as tf
import nltk
stop_words = stopwords.words('english')

In [74]:
from re import sub

from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pradyut.singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pradyut.singh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
stop_words = stopwords.words('english')

In [4]:
dataset = joblib.load("C:\AmazonHack\compressed") 

In [76]:
dataset_copy = dataset.copy(deep = True)

In [77]:
dataset_copy.head()

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll, Based ...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,[Skyhorse Publishing],imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids, Chewable Iron Tablets, Quanti...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,[OTHERS],OTHERS,4


In [78]:
dataset.head()

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll, Based ...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,[Skyhorse Publishing],imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids, Chewable Iron Tablets, Quanti...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,[OTHERS],OTHERS,4


In [79]:
dataset.shape

(2801530, 4)

In [50]:
dataset_temp = dataset_copy.iloc[:10000,:]

In [20]:
dataset_temp.head()

NameError: name 'dataset_temp' is not defined

ML Approach


In [52]:
X = dataset_temp['TITLE']
Y = dataset_temp['BROWSE_NODE_ID']

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [58]:
X_train.shape

(7000,)

In [60]:
X_test.shape

(3000,)

In [66]:
nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', MultinomialNB())])

In [67]:
y_pred = nb.predict(X_test)

In [72]:
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.138
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00        40
           5       0.18      0.92      0.30        66
           6       0.00      0.00      0.00         5
           7       0.00      0.00      0.00         2
          10       0.00      0.00      0.00         1
          11       1.00      0.83      0.91        24
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
          17       0.00      0.00      0.00         1
          19       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         1
          22       1.00      0.06      0.11        17
          23       0.00      0.00      0.00         2
          28

# DL Approach

In [25]:
embeddings_index = {}
try:
    
    f = open('C:\AmazonHack\glove.840B.300d.txt', encoding = "utf8")
    for line in tqdm(f):
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    print('Found %s word vectors.' % len(embeddings_index))
    
except:
    pass

52343it [00:05, 9782.89it/s] 


In [51]:
def sent2vec(s):
    words = str(s).lower()
    words = word_tokenize(words)
    words = [w for w in words if not w in stop_words]
    words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [52]:
dataset_temp = dataset.iloc[:20000,:].copy()
dataset_temp

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll, Based ...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,[Skyhorse Publishing],imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids, Chewable Iron Tablets, Quanti...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,[OTHERS],OTHERS,4
...,...,...,...,...
20008,Puma Men's Hybrid Runner Desert Whisper White-...,"[Outer Material: Textile, Closure Type: Lace-U...",Puma,701
20009,Shining Jewel 24K Gold Plated Panch Mukhi Hanu...,[High Quality Bajarang Bali Gold Plated Coin N...,Shining Jewel,1193
20010,Casa Decor Forest Branch Textured Metal Iron S...,[DESIGN : The eclectic and nature inspired des...,Casa Décor,12558
20011,Tradico® TradicoBrand New 114mm to 89mm Pipe O...,[OTHERS],OTHERS,4


In [54]:
y = dataset_temp['BROWSE_NODE_ID'].values
y

array([    0,     1,     2, ..., 12558,     4,   876], dtype=int64)

In [55]:
y = np.reshape(y.copy(), (20000, 1))

In [57]:
y.shape

(20000, 1)

In [60]:
y_enc = tf.keras.utils.to_categorical(y.copy())
y_enc.shape

(20000, 19999)

In [61]:
X_train, X_test, y_train_will_check, y_test_will_check = train_test_split(dataset_temp['TITLE'], y.copy(), shuffle = False, random_state = 0)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(dataset_temp['TITLE'], y_enc, shuffle = False, random_state = 0)

In [63]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(15000,)
(5000,)
(15000, 19999)
(5000, 19999)


In [64]:
xtrain_glove = [sent2vec(x) for x in tqdm(X_train.copy())]
xtest_glove = [sent2vec(x) for x in tqdm(X_test.copy())]

xtrain_glove = np.array(xtrain_glove)
xtest_glove = np.array(xtest_glove)

print(xtrain_glove.shape)
print(xtest_glove.shape)
print(y_train.shape)
print(y_test.shape)

scl = preprocessing.StandardScaler()
xtrain_glove_scl = scl.fit_transform(xtrain_glove)
xtest_glove_scl = scl.transform(xtest_glove)

100%|████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:01<00:00, 2941.00it/s]


(15000, 300)
(5000, 300)
(15000, 19999)
(5000, 19999)


In [67]:
model = Sequential()
print('Model Initialised')

model.add(Dense(300, input_dim=300, activation='relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
print('3 layers added')

model.add(Dense(300, activation='relu'))
model.add(Dropout(0.3))
model.add(BatchNormalization())
print('3 layers added')

model.add(Dense(19999))
model.add(Activation('softmax'))
print('Softmax added')

# compile the model
print('Model compiling')
model.compile(loss='categorical_crossentropy', optimizer='adam')
print('DONE')

Model Initialised
3 layers added
3 layers added
Softmax added
Model compiling
DONE


In [68]:
model.fit(xtrain_glove_scl, y=y_train, batch_size=64, 
          epochs=2, verbose=2, 
          validation_data=(xtest_glove_scl, y_test))

Epoch 1/2
235/235 - 24s - loss: 6.5942 - val_loss: 5.4790
Epoch 2/2
235/235 - 20s - loss: 3.9099 - val_loss: 4.9666


In [69]:
y_pred = model.predict_classes(xtest_glove_scl)

In [73]:
y_test_will_check.shape

(5000, 1)

In [71]:
y_pred

array([  55, 2813, 1045, ...,   74,    4,  803], dtype=int64)

In [75]:
# y_pred = model.predict(xtest_glove_scl)
# print('accuracy %s' % accuracy_score(y_pred, y_train))

## Another approach

In [88]:
dataset_temp = dataset.iloc[:20000,:].copy()
dataset_temp

,TITLE,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","[Pete the Cat Bedtime Blues plush doll, Based ...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,[Skyhorse Publishing],imusti,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),"[Nutrilite Kids, Chewable Iron Tablets, Quanti...",Amway,3
4,Teacher Planner Company A4 6 Lesson Academic T...,[OTHERS],OTHERS,4
...,...,...,...,...
20008,Puma Men's Hybrid Runner Desert Whisper White-...,"[Outer Material: Textile, Closure Type: Lace-U...",Puma,701
20009,Shining Jewel 24K Gold Plated Panch Mukhi Hanu...,[High Quality Bajarang Bali Gold Plated Coin N...,Shining Jewel,1193
20010,Casa Decor Forest Branch Textured Metal Iron S...,[DESIGN : The eclectic and nature inspired des...,Casa Décor,12558
20011,Tradico® TradicoBrand New 114mm to 89mm Pipe O...,[OTHERS],OTHERS,4


In [110]:
df = dataset_temp[['TITLE','BROWSE_NODE_ID']].copy()

In [111]:
df

,TITLE,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch",0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",1
2,The Ultimate Self-Sufficiency Handbook: A Comp...,2
3,Amway Nutrilite Kids Chewable Iron Tablets (100),3
4,Teacher Planner Company A4 6 Lesson Academic T...,4
...,...,...
20008,Puma Men's Hybrid Runner Desert Whisper White-...,701
20009,Shining Jewel 24K Gold Plated Panch Mukhi Hanu...,1193
20010,Casa Decor Forest Branch Textured Metal Iron S...,12558
20011,Tradico® TradicoBrand New 114mm to 89mm Pipe O...,4


In [112]:
def remove_URL(text):
    url = re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"", text)

def remove_punct(text):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [113]:
pattern = re.compile(r"https?://(\S+|www)\.\S+")
for t in df['TITLE']:
    matches = pattern.findall(t)
    for match in matches:
        print(t)
        print(match)
        print(pattern.sub(r"", t))
    if len(matches) > 0:
        break

In [114]:
df['TITLE'] = df['TITLE'].map(remove_URL)
df['TITLE'] = df['TITLE'].map(remove_punct)

In [115]:
stop = set(stopwords.words("english"))
def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

In [116]:
df['TITLE'] = df['TITLE'].map(remove_stopwords)

In [117]:
df['TITLE']

0                     pete cat bedtime blues doll 145 inch
1            new yorker nyhm014 refrigerator magnet 2 x 35
2        ultimate selfsufficiency handbook complete gui...
3           amway nutrilite kids chewable iron tablets 100
4        teacher planner company a4 6 lesson academic t...
                               ...                        
20008    puma mens hybrid runner desert whisper whiteme...
20009    shining jewel 24k gold plated panch mukhi hanu...
20010    casa decor forest branch textured metal iron s...
20011    tradico® tradicobrand new 114mm 89mm pipe od 4...
20012    youbella blue crystal necklace earrings women ...
Name: TITLE, Length: 20000, dtype: object

In [118]:
from collections import Counter

def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter = counter_word(df['TITLE'])

In [119]:
len(counter)

39160

In [120]:
counter.most_common(5)

[('cover', 2164),
 ('womens', 2050),
 ('mens', 1946),
 ('black', 1873),
 ('cotton', 1599)]

In [121]:
num_unique_words = len(counter)

In [122]:
num_unique_words

39160

In [124]:
train_size = int(df.shape[0] * 0.8)

train_df = df[:train_size]
val_df = df[train_size:]

# split text and labels
train_sentences = train_df['TITLE'].to_numpy()
train_labels = train_df['BROWSE_NODE_ID'].to_numpy()
val_sentences = val_df['TITLE'].to_numpy()
val_labels = val_df['BROWSE_NODE_ID'].to_numpy()

In [125]:
train_sentences.shape, val_sentences.shape

((16000,), (4000,))

In [126]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(train_sentences) 

In [127]:
word_index = tokenizer.word_index

In [128]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
val_sequences = tokenizer.texts_to_sequences(val_sentences)

In [129]:
print(train_sentences[10:15])
print(train_sequences[10:15])

['sloniacin polygel controlledrelease niacin 750 mg 100 tablets'
 'white braun kf400wh aromaster 10cup coffeemaker white'
 'yamaha pcmskit portable electronic keyboard adapter pc joystick midi cable'
 'microsoft sidewinder gamepad pro'
 'jiffy steamer plastic steam head beige 120v']
[[10872, 10873, 10874, 10875, 998, 1550, 57, 1104], [15, 7239, 10876, 10877, 10878, 10879, 15], [1245, 10880, 232, 822, 513, 178, 233, 3548, 1412, 107], [1955, 10881, 3162, 48], [10882, 2839, 71, 2582, 354, 294, 10883]]


In [130]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Max number of words in a sequence
max_length = 20

train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post", truncating="post")
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding="post", truncating="post")
train_padded.shape, val_padded.shape

((16000, 20), (4000, 20))

In [131]:
print(train_sentences[10])
print(train_sequences[10])
print(train_padded[10])

sloniacin polygel controlledrelease niacin 750 mg 100 tablets
[10872, 10873, 10874, 10875, 998, 1550, 57, 1104]
[10872 10873 10874 10875   998  1550    57  1104     0     0     0     0
     0     0     0     0     0     0     0     0]


In [133]:
from tensorflow.keras import layers
model = tf.keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32, input_length=max_length))

model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 32)            1253120   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 1,278,017
Trainable params: 1,278,017
Non-trainable params: 0
_________________________________________________________________


In [135]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)
optim = tf.keras.optimizers.Adam(lr=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [136]:
model.fit(train_padded, train_labels, epochs=20, validation_data=(val_padded, val_labels), verbose=1)

Epoch 1/20
500/500 [==============================] - 15s 27ms/step - loss: -53838.8906 - accuracy: 4.3750e-04 - val_loss: -156255.9062 - val_accuracy: 5.0000e-04
Epoch 2/20
500/500 [==============================] - 13s 26ms/step - loss: -123283.0781 - accuracy: 4.3750e-04 - val_loss: -271077.8438 - val_accuracy: 5.0000e-04
Epoch 3/20
500/500 [==============================] - 13s 26ms/step - loss: -189229.7031 - accuracy: 4.3750e-04 - val_loss: -384775.4688 - val_accuracy: 5.0000e-04
Epoch 4/20
500/500 [==============================] - 13s 27ms/step - loss: -254542.7656 - accuracy: 4.3750e-04 - val_loss: -497712.5938 - val_accuracy: 5.0000e-04
Epoch 5/20
500/500 [==============================] - 15s 30ms/step - loss: -319765.6875 - accuracy: 4.3750e-04 - val_loss: -610790.1250 - val_accuracy: 5.0000e-04
Epoch 6/20
500/500 [==============================] - 14s 28ms/step - loss: -385061.7188 - accuracy: 4.3750e-04 - val_loss: -723878.6250 - val_accuracy: 5.0000e-04
Epoch 7/20
500/50